In [7]:
!pip install sparqlwrapper
!python3 -m pip install pandas

    100% |████████████████████████████████| 9.5MB 199kB/s  eta 0:00:01


In [25]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get tanks
query = """SELECT DISTINCT ?tank ?tankLabel ?image WHERE {
  ?tank   wdt:P31 wd:Q920182;
          rdfs:label ?tankLabel;
          wdt:P18 ?image.
 SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
 FILTER(lang(?tankLabel)="en")
}
LIMIT 10000"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append((result["tankLabel"]["value"], result["image"]["value"]))

In [26]:
import urllib.request
import json
import pandas as pd

dataframe = pd.DataFrame(array, columns=["planet", "image"])
dataframe = dataframe.astype(
    dtype={"planet": "<U200", "image": "<U200"}
)
dataframe

,planet,image
0,Olifant tanks,http://commons.wikimedia.org/wiki/Special:File...
1,TR-125,http://commons.wikimedia.org/wiki/Special:File...
2,T-72M4 CZ,http://commons.wikimedia.org/wiki/Special:File...
3,VT tank,http://commons.wikimedia.org/wiki/Special:File...
4,Leopard 2E,http://commons.wikimedia.org/wiki/Special:File...
...,...,...
59,T-72 operators and variants,http://commons.wikimedia.org/wiki/Special:File...
60,Obiekt 292,http://commons.wikimedia.org/wiki/Special:File...
61,Type 99A tank,http://commons.wikimedia.org/wiki/Special:File...
62,Panther KF51,http://commons.wikimedia.org/wiki/Special:File...


In [57]:
import requests
import shutil
import os
import hashlib

def download_image(url):
    
    # Le dossier dans lequel enregistrer les images
    directory = "./images"

    # Création du dossier s'il n'existe pas encore
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    headers = {"User-Agent":"Mozilla/5.0"}
    request = requests.get(url,
        allow_redirects=True,
        headers=headers,
        stream=True)
    
    try:
        request = requests.get(url, stream=True)
        request.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    

    if request.status_code == 200:
        filename = hashlib.md5(url.encode('utf-8')).hexdigest() + ".jpg"
        filepath = os.path.join(directory, filename)

        with open(filepath, 'wb') as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
            
    return request.status_code

In [59]:
dataframe.image.apply(download_image)

HTTP error occurred: 403 Client Error: Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy for url: https://upload.wikimedia.org/wikipedia/commons/7/7c/Romanian_T-72M_tank.jpg
HTTP error occurred: 403 Client Error: Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy for url: https://upload.wikimedia.org/wikipedia/commons/8/86/VT1-2-2.jpg
HTTP error occurred: 403 Client Error: Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy for url: https://upload.wikimedia.org/wikipedia/commons/0/02/%E9%B3%84%E9%B1%BC%E6%B9%96%E7%9A%84%E5%9D%A6%E5%85%8B_-_panoramio.jpg
HTTP error occurred: 403 Client Error: Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy for url: https://upload.wikimedia.org/wikipedia/commons/4/48/AMX32.jpg
HTTP error occurred: 403 Client Error: Forbidden. Please comply with th

0     200.0
1       NaN
2     200.0
3       NaN
4     200.0
      ...  
59    200.0
60      NaN
61      NaN
62      NaN
63    200.0
Name: image, Length: 64, dtype: float64